### Setup my scraper

In [2]:
# 1- import liberaries
import requests
from bs4 import BeautifulSoup
import csv
import lxml
from itertools import zip_longest

### find the element containig what we need
job title, job skills, company name, location name

In [3]:
job_title = []
job_skill = []
company_name = []
location = []
links = []
page_num = 0

In [25]:
while True :
    # 2- requests to fetch the url
    url = f'https://wuzzuf.net/search/jobs/?a=navbl&q=python&start={page_num}'
    response = requests.get(url)
    
    # 3- page content
    page = response.content 
    
    # 4- creat soup object to parse content
    soup = BeautifulSoup(page, 'lxml')
    
    page_limit = int(soup.find("strong").text)
    
    if (page_num > page_limit // 15):
        break
    
    
    job_titles = soup.find_all('h2',{'class':'css-m604qf'})
    job_skills = soup.find_all('div',{'class':'css-y4udm8'})
    company_names = soup.find_all('a', {"class":"css-17s97q8"})
    locations = soup.find_all('span', {"class":"css-5wys0k"})
    
    
   

    for i in range(len(job_titles)):
        job_title.append(job_titles[i].text)
    
        links.append(job_titles[i].find('a').attrs['href'])
        
    
        job_skill.append(job_skills[i].text)
        company_name.append(company_names[i].text)
        location.append(locations[i].text)

    page_num += 1
    

### Creat Dataframe

In [39]:
import pandas as pd

df = pd.DataFrame({"Job Titles":job_title,
                   "Job Skills":job_skill,
                   "Company Name":company_name,
                   "Location":location,
                   "Links":links})
df

,Job Titles,Job Skills,Company Name,Location,Links
0,Senior Java/ Scala Backend Developer with Pyth...,Full TimeWork From HomeExperienced · 5+ Yrs of...,GetTechForce.com -,"Cairo, Egypt",/jobs/p/jOSpVK9DRTCc-Senior-Java-Scala-Backend...
1,Senior Python Developer,Full TimeExperienced · 4+ Yrs of Exp · IT/Soft...,Trufla -,"Heliopolis, Cairo, Egypt",/jobs/p/WBHqaf7WeZYe-Senior-Python-Developer-T...
2,Python Developer,Freelance / ProjectManager · 4 - 7 Yrs of Exp ...,Synergy Markets -,"New Cairo, Cairo, Egypt",/jobs/p/Coiipd7kP6Za-Python-Developer-Synergy-...
3,Software Engineer (Python),Full TimePart TimeWork From HomeExperienced · ...,Tam Development LLC -,"Riyadh, Saudi Arabia",/jobs/p/rdnYmzyFCkGJ-Software-Engineer-Python-...
4,Python and Javascript Machine Learning Enginee...,Full TimeWork From HomeExperienced · 3+ Yrs of...,GetTechForce.com -,"Cairo, Egypt",/jobs/p/DgaOYxVHPUE7-Python-and-Javascript-Mac...
...,...,...,...,...,...
113,IT Technical Support (Work Will Be Allocated I...,Full TimeExperienced · 5 - 15 Yrs of Exp · Ins...,Confidential -,"Cairo, Egypt",/jobs/p/rKayHyYA4XAD-IT-Technical-Support-Work...
114,Full Stack Developer,Full TimeExperienced · 2+ Yrs of Exp · IT/Soft...,Paymint -,"Cairo, Egypt",/jobs/p/PPTyC61x9NPl-Full-Stack-Developer-Paym...
115,Kotlin Developer - FinTech,Full TimeExperienced · 4+ Yrs of Exp · IT/Soft...,Bright Creations -,"Smart Village, Giza, Egypt",/jobs/p/tD1HfvrFFkiV-Kotlin-Developer---FinTec...
116,Junior Software Developer,Full TimeEntry Level · 0 - 2 Yrs of Exp · IT/S...,Amriya Pharmaceutical Industries -,"Ameria, Alexandria, Egypt",/jobs/p/6mxsu38sp272-Junior-Software-Developer...


### Data Cleaning

In [40]:
df["Links"] = ['https://wuzzuf.net'+l for l in df["Links"] ]
df['Company Name'] = [name.replace('-','') for name in df['Company Name']]
df['Job Skills'] = [skills.replace('·',",") for skills in df['Job Skills']]

In [41]:
df["Links"]

0      https://wuzzuf.net/jobs/p/jOSpVK9DRTCc-Senior-...
1      https://wuzzuf.net/jobs/p/WBHqaf7WeZYe-Senior-...
2      https://wuzzuf.net/jobs/p/Coiipd7kP6Za-Python-...
3      https://wuzzuf.net/jobs/p/rdnYmzyFCkGJ-Softwar...
4      https://wuzzuf.net/jobs/p/DgaOYxVHPUE7-Python-...
                             ...                        
113    https://wuzzuf.net/jobs/p/rKayHyYA4XAD-IT-Tech...
114    https://wuzzuf.net/jobs/p/PPTyC61x9NPl-Full-St...
115    https://wuzzuf.net/jobs/p/tD1HfvrFFkiV-Kotlin-...
116    https://wuzzuf.net/jobs/p/6mxsu38sp272-Junior-...
117    https://wuzzuf.net/jobs/p/Tiaz4nYviy40-Java-De...
Name: Links, Length: 118, dtype: object

In [42]:
df['Job Skills'][0]

'Full TimeWork From HomeExperienced , 5+ Yrs of Exp , IT/Software Development , Engineering - Telecom/Technology , AngularJS , Computer Science , Design , Development , HTML5 , Information Technology (IT) , JPA , REST'

### Save data at csv file

In [44]:
file = df.to_csv("Wzzuf_python_jobs.csv")